In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc

# try:
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Dense
#     from tensorflow.keras.optimizers import Adam
#     from tensorflow.keras.metrics import Precision, Recall, MeanSquaredError, AUC
# except ImportError:
#     from keras.models import Sequential
#     from keras.layers import Dense
#     from keras.optimizers import Adam
#     from keras.metrics import Precision, Recall, MeanSquaredError, AUC

# # Carregar os dados
# data = pd.read_csv('HIGGS_100k.csv', header=None, names=[
#     'class', 'lepton pT', 'lepton eta', 'lepton phi', 'missing energy magnitude', 
#     'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 
#     'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 
#     'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag', 
#     'm jj', 'm jjj', 'm lv', 'm jlv', 'm bb', 'm wbb', 'm wwbb'
# ])

# # Separar as features de baixo nível e o rótulo
# label = data['class']
# low_level = data.iloc[:, 1:22]

# # Normalizar os dados
# scaler = StandardScaler()
# low_level_scaled = scaler.fit_transform(low_level)

# # Dividir os dados em treino e teste
# X_train, X_test, y_train, y_test = train_test_split(low_level_scaled, label, random_state=42, test_size=0.25)

# # Função para plotar o histórico de treinamento
# def plot_epoch(history, metric):
#     plt.figure(figsize=(10, 6))
#     plt.plot(history.history[f'val_{metric}'], color='blue', label=f"Validação {metric}")
#     plt.plot(history.history[metric], color='red', label=f"Treinamento {metric}")
#     plt.title(f"Histórico de treinamento: {metric.capitalize()}")
#     plt.xlabel("Épocas")
#     plt.ylabel(metric.capitalize())
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# # Definição do novo modelo de rede neural
# new_model = Sequential([
#     Dense(128, activation='tanh', input_shape=(21,)),
#     Dense(64, activation='tanh'),
#     Dense(32, activation='tanh'),
#     Dense(16, activation='tanh'),
#     Dense(1, activation='sigmoid')
# ])

# # Compilar o novo modelo
# new_optimizer = Adam(learning_rate=0.0001)
# new_model.compile(optimizer=new_optimizer, loss='binary_crossentropy', 
#                   metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), MeanSquaredError(name='mean_squared_error'), AUC(name='auc')])

# # Treinar o novo modelo
# new_epochs = 100
# new_batch_size = 128

# new_history = new_model.fit(X_train, y_train, epochs=new_epochs, validation_data=(X_test, y_test), batch_size=new_batch_size)

# # Plotar o histórico de treinamento do novo modelo
# plot_epoch(new_history, 'accuracy')
# plot_epoch(new_history, 'precision')
# plot_epoch(new_history, 'recall')
# plot_epoch(new_history, 'mean_squared_error')
# plot_epoch(new_history, 'auc')

# # Previsões do novo modelo
# new_predictions = new_model.predict(low_level_scaled)

# # Aplicar limiar para classificar como sinal ou ruído
# new_predictions = (new_predictions > 0.5).astype(int)

# # Adicionar previsões ao dataframe original
# data['new_pred'] = new_predictions

# # Contar a porcentagem de cada classe nas previsões
# print(data['new_pred'].value_counts(normalize=True) * 100)

# # Separar os dados em sinal e ruído com base nas previsões e rótulos originais
# new_dados_sinal = data[data['class'] == 1]
# new_dados_ruido = data[data['class'] == 0]
# new_sinal_pred = data[data['new_pred'] == 1]
# new_ruido_pred = data[data['new_pred'] == 0]

# # Plotar histogramas
# plt.figure(figsize=(14, 8))
# plt.hist(new_dados_sinal['m wwbb'], bins=100, alpha=0.6, label='Sinal (original)', color='blue', edgecolor='black')
# plt.hist(new_dados_ruido['m wwbb'], bins=100, alpha=0.6, label='Ruído (original)', color='red', edgecolor='black')
# plt.hist(new_sinal_pred['m wwbb'], bins=100, alpha=0.6, label='Sinal (rede neural)', color='green', edgecolor='black')
# plt.hist(new_ruido_pred['m wwbb'], bins=100, alpha=0.6, label='Ruído (rede neural)', color='orange', edgecolor='black')

# plt.legend()
# plt.xlabel('m wwbb')
# plt.ylabel('Frequência normalizada')
# plt.title('Distribuição de m wwbb para Sinal e Ruído')
# plt.grid(True)
# plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc

try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.metrics import Precision, Recall, MeanSquaredError, AUC
except ImportError:
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.optimizers import Adam
    from keras.metrics import Precision, Recall, MeanSquaredError, AUC

# Carregar um subconjunto dos dados
data = pd.read_csv('HIGGS_100k.csv', header=None, names=[
    'class', 'lepton pT', 'lepton eta', 'lepton phi', 'missing energy magnitude', 
    'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 
    'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 
    'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag', 
    'm jj', 'm jjj', 'm lv', 'm jlv', 'm bb', 'm wbb', 'm wwbb'
])

# Separar as features de baixo nível e o rótulo
label = data['class']
low_level = data.iloc[:, 1:22]

# Normalizar os dados
scaler = StandardScaler()
low_level_scaled = scaler.fit_transform(low_level)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(low_level_scaled, label, random_state=42, test_size=0.25)

# Função para plotar o histórico de treinamento
def plot_epoch(history, metric):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history[f'val_{metric}'], color='blue', label=f"Validação {metric}")
    plt.plot(history.history[metric], color='red', label=f"Treinamento {metric}")
    plt.title(f"Histórico de treinamento: {metric.capitalize()}")
    plt.xlabel("Épocas")
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)
    plt.show()

# Definição do novo modelo de rede neural
new_model = Sequential([
    Dense(64, activation='tanh', input_shape=(21,)),
    Dense(32, activation='tanh'),
    Dense(16, activation='tanh'),
    Dense(1, activation='sigmoid')
])

# Compilar o novo modelo
new_optimizer = Adam(learning_rate=0.0001)
new_model.compile(optimizer=new_optimizer, loss='binary_crossentropy', 
                  metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), MeanSquaredError(name='mean_squared_error'), AUC(name='auc')])

# Treinar o novo modelo
new_epochs = 50
new_batch_size = 64

new_history = new_model.fit(X_train, y_train, epochs=new_epochs, validation_data=(X_test, y_test), batch_size=new_batch_size)

# Plotar o histórico de treinamento do novo modelo
plot_epoch(new_history, 'accuracy')
plot_epoch(new_history, 'precision')
plot_epoch(new_history, 'recall')
plot_epoch(new_history, 'mean_squared_error')
plot_epoch(new_history, 'auc')

# Previsões do novo modelo
new_predictions = new_model.predict(low_level_scaled)

# Aplicar limiar para classificar como sinal ou ruído
new_predictions = (new_predictions > 0.5).astype(int)

# Adicionar previsões ao dataframe original
data['new_pred'] = new_predictions

# Contar a porcentagem de cada classe nas previsões
print(data['new_pred'].value_counts(normalize=True) * 100)

# Separar os dados em sinal e ruído com base nas previsões e rótulos originais
new_dados_sinal = data[data['class'] == 1]
new_dados_ruido = data[data['class'] == 0]
new_sinal_pred = data[data['new_pred'] == 1]
new_ruido_pred = data[data['new_pred'] == 0]

# Plotar histogramas
plt.figure(figsize=(14, 8))
plt.hist(new_dados_sinal['m wwbb'], bins=100, alpha=0.6, label='Sinal (original)', color='blue', edgecolor='black')
plt.hist(new_dados_ruido['m wwbb'], bins=100, alpha=0.6, label='Ruído (original)', color='red', edgecolor='black')
plt.hist(new_sinal_pred['m wwbb'], bins=100, alpha=0.6, label='Sinal (rede neural)', color='green', edgecolor='black')
plt.hist(new_ruido_pred['m wwbb'], bins=100, alpha=0.6, label='Ruído (rede neural)', color='orange', edgecolor='black')

plt.legend()
plt.xlabel('m wwbb')
plt.ylabel('Frequência normalizada')
plt.title('Distribuição de m wwbb para Sinal e Ruído')
plt.grid(True)
plt.show()
